In [11]:
import numpy as np
import pandas as pd
from datetime import datetime

In [12]:
import sys
import psycopg2
import pickle
import csv


#==============================================================================
# Main function
#==============================================================================
def main(argv):
    tbl = 'actransit'
    conn = psycopg2.connect("dbname=transit user=postgres", password="postgres")
    cur = conn.cursor()
    with open("actransit_route_timeseries_2019.csv",'w', newline='') as f:
        writer = csv.writer(f, dialect='excel')
        writer.writerow(['Route', 'Date', 'Hour', 'Passenger on', 'Passenger off','Passenger Load'])
        stmt = "SELECT route,CAST(act_stop_time as DATE) as StartDate, DATE_PART('hour', act_stop_time),SUM(psgr_on) as passenger_on, SUM(psgr_off) as passenger_off, SUM(psgr_load) as passenger_load from "
        stmt += tbl
        stmt += " WHERE act_stop_time >= TO_DATE('2019-09-01', 'YYYY-MM-DD') AND act_stop_time <=  TO_DATE('2019-12-31', 'YYYY-MM-DD')"
        stmt += " GROUP BY route, CAST(act_stop_time as DATE), DATE_PART('hour', act_stop_time) "
        stmt += " ORDER BY CAST(act_stop_time as DATE) ASC;"

        cur.execute(stmt)

        result = cur.fetchall()
        for record in result:
            writer.writerow([record[0], record[1], record[2], record[3], record[4], record[5]])

    conn.commit()
    cur.close()
    conn.close()


if __name__ == "__main__":
    main(sys.argv)

